In [1]:
import numpy as np
import pandas as pd
from gurobipy import GRB
import gurobipy as gp
import random
import networkx as nx
import math

In [2]:
NodeCount = 8
GraphDensityCoefficient = 0.5
MaxCostArc = 20
RequiredInfo = 20
CommodityCount = 5
MaxDemandCommodity = 20
MaxCapacityArc = 20

random.seed(10)

N = [i+1 for i in range(NodeCount)]
Commodities = [i+1 for i in range(CommodityCount)]
NodeNodeAdjacencyMatrix = np.zeros((NodeCount,NodeCount))


ArcProbability = np.zeros((NodeCount,NodeCount))

for i in range(NodeCount-1):
    for j in range(i+1,NodeCount):
        ArcProbability[i,j] = random.uniform(0,1)
        if ArcProbability[i,j] > GraphDensityCoefficient:
            
            NodeNodeAdjacencyMatrix[i,j] = 1
            
    if sum(NodeNodeAdjacencyMatrix[i]) == 0:
        
        NodeNodeAdjacencyMatrix[i,NodeCount-1] = 1

ArcCount = int(sum(sum(NodeNodeAdjacencyMatrix)))



In [3]:
NodeNodeAdjacencyMatrix

array([[0., 1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 0., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
ArcCount    

18

In [5]:
# G = nx.from_numpy_array(NodeNodeAdjacencyMatrix,create_using=nx.DiGraph())
# nx.draw_circular(G,arrows=True)

In [6]:
NodeNodeAdjacencyList = {i:[] for i in N}
CostDict = {}
CapacityDict = {}
ArcDict = {i:0 for i in range(ArcCount)}
InfoDict = {i:0 for i in range(ArcCount)}

ArcCounter = 0

for i in range(NodeCount):
    for j in range(NodeCount):
        if NodeNodeAdjacencyMatrix[i][j] == 1:
            NodeNodeAdjacencyList[i+1].append(j+1)
            CostDict[ArcCounter]= random.randint(1,MaxCostArc)
            CapacityDict[ArcCounter] = random.randint(MaxCapacityArc/2,MaxCapacityArc)
            InfoDict[ArcCounter] = random.randint(0,RequiredInfo/4)
            ArcDict[ArcCounter] = (i+1,j+1)
            ArcCounter += 1

In [7]:
ArcDict

{0: (1, 2),
 1: (1, 4),
 2: (1, 6),
 3: (1, 7),
 4: (1, 8),
 5: (2, 4),
 6: (2, 7),
 7: (2, 8),
 8: (3, 5),
 9: (3, 6),
 10: (4, 5),
 11: (4, 7),
 12: (4, 8),
 13: (5, 7),
 14: (5, 8),
 15: (6, 7),
 16: (6, 8),
 17: (7, 8)}

In [8]:
NodeNodeAdjacencyList

{1: [2, 4, 6, 7, 8],
 2: [4, 7, 8],
 3: [5, 6],
 4: [5, 7, 8],
 5: [7, 8],
 6: [7, 8],
 7: [8],
 8: []}

In [9]:
InfoDict

{0: 1,
 1: 4,
 2: 2,
 3: 3,
 4: 5,
 5: 4,
 6: 3,
 7: 0,
 8: 4,
 9: 0,
 10: 4,
 11: 1,
 12: 3,
 13: 2,
 14: 3,
 15: 2,
 16: 4,
 17: 2}

In [10]:
CostDict

{0: 1,
 1: 7,
 2: 12,
 3: 18,
 4: 16,
 5: 19,
 6: 6,
 7: 8,
 8: 16,
 9: 3,
 10: 5,
 11: 12,
 12: 4,
 13: 6,
 14: 14,
 15: 8,
 16: 5,
 17: 6}

In [11]:
CapacityDict

{0: 13,
 1: 14,
 2: 13,
 3: 17,
 4: 11,
 5: 15,
 6: 13,
 7: 10,
 8: 14,
 9: 18,
 10: 16,
 11: 19,
 12: 11,
 13: 13,
 14: 16,
 15: 20,
 16: 19,
 17: 11}

In [12]:
B_k = [random.randint(1,MaxDemandCommodity) for i in range(CommodityCount)]
B_k

[15, 10, 6, 6, 6]

In [13]:
Origin = [random.randint(1,math.floor(NodeCount/2)) for i in range(CommodityCount)]
Destination = [random.randint(math.floor(NodeCount/2)+1,NodeCount) for i in range(CommodityCount)]

In [14]:
Origin

[4, 3, 3, 4, 2]

In [15]:
Destination

[5, 5, 7, 7, 6]

In [16]:
def Reachable(visited, graph, node): 
    visited.append(node)
    queue.append(node)
    Reachable = []

    while queue:          
        m = queue.pop(0) 
#         print (m, end = " ") 
        Reachable.append(m)

        for neighbour in graph[m]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return Reachable

In [17]:
for i in range(len(Origin)):
    visited = [] # List for visited nodes.
    queue = []     #Initialize a queue
    ReachableList = Reachable(visited,NodeNodeAdjacencyList,Origin[i])
#     print(i,Origin[i],Destination[i],ReachableList)
    k = len(ReachableList)

# #             break
    if Destination[i] not in ReachableList:
        for j in range(k):
            if ReachableList[j]>Origin[i]:
                Destination[i] = ReachableList[j]
                
        

In [18]:
Destination

[5, 5, 7, 7, 5]

In [19]:
B_k_Dict = {i:[] for i in Commodities}
Origin_Dict = {i:[] for i in Commodities}
Destination_Dict = {i:[] for i in Commodities}
for i in range(CommodityCount):
    B_k_Dict[i+1]=B_k[i]
    Origin_Dict[i+1] = Origin[i]
    Destination_Dict[i+1] = Destination[i]

In [20]:
NodeNodeAdjacencyListWithDummyArcs = {i:[] for i in N}
# PathCost = {}
# CapacityListInitial = {}
InitialHighCost = MaxCostArc*ArcCount + 1
InitialHighCapacity = MaxDemandCommodity*ArcCount + 1

for i in range(len(Origin)):

    NodeNodeAdjacencyListWithDummyArcs[Origin[i]].append(Destination[i])

NodeNodeAdjacencyListWithDummyArcs

{1: [], 2: [5], 3: [5, 7], 4: [5, 7], 5: [], 6: [], 7: [], 8: []}

In [21]:
PathSetForCommodity = {i:[] for i in Commodities}
Paths = [i for i in range(CommodityCount)]

for i in range(CommodityCount):
    PathSetForCommodity[i+1].append(Paths[i])
PathSetForCommodity

{1: [0], 2: [1], 3: [2], 4: [3], 5: [4]}

In [22]:
Paths

[0, 1, 2, 3, 4]

In [23]:
PathDict = {i:[] for i in Paths}
NodesInPath = {i:[] for i in Paths}
for i in Paths:
    PathDict[i].append((Origin[i],Destination[i]))
    NodesInPath[i].append(Origin[i])
    NodesInPath[i].append(Destination[i])
PathDict

{0: [(4, 5)], 1: [(3, 5)], 2: [(3, 7)], 3: [(4, 7)], 4: [(2, 5)]}

In [24]:
NodesInPath

{0: [4, 5], 1: [3, 5], 2: [3, 7], 3: [4, 7], 4: [2, 5]}

In [25]:
PathCost = {p:InitialHighCost for p in Paths}
PathInfo = {p:RequiredInfo for p in Paths}
for i in Paths:
    for j in range(ArcCount):
        if list(ArcDict.values())[j] == (PathDict[i][0]):
            ArcIndex = list(ArcDict.values())[j]
            PathCost[i] = CostDict[j]
            PathInfo[i] = InfoDict[j]
# for i in Paths:
#     if PathCost[i] == InitialHighCost:
#         PathInfo[i] = RequiredInfo

        
#         else:
#             PathCost[i] = InitialHighCost
    
Counter = 0

for i in N:
    for j in N:
        if (i in Origin) and (j in Destination):
            
            if NodeNodeAdjacencyMatrix[i-1,j-1] == 0:
                CapacityDict[ArcCount+Counter] = InitialHighCapacity
                CostDict[ArcCount+Counter] = InitialHighCost
                InfoDict[ArcCount+Counter] = RequiredInfo
                NodeNodeAdjacencyMatrix[i-1,j-1] = 1
                ArcDict[ArcCount+Counter] = (i,j)
                
                Counter += 1
                
#             else:
#                 PathCost[(i,j)] = CostList[(i,j)]
ArcCountWithDummyArcs = ArcCount + Counter

In [26]:
PathInfo

{0: 4, 1: 4, 2: 20, 3: 1, 4: 20}

In [27]:
PathCost

{0: 5, 1: 16, 2: 361, 3: 12, 4: 361}

In [28]:
CapacityDict

{0: 13,
 1: 14,
 2: 13,
 3: 17,
 4: 11,
 5: 15,
 6: 13,
 7: 10,
 8: 14,
 9: 18,
 10: 16,
 11: 19,
 12: 11,
 13: 13,
 14: 16,
 15: 20,
 16: 19,
 17: 11,
 18: 361,
 19: 361}

In [29]:
InfoDict

{0: 1,
 1: 4,
 2: 2,
 3: 3,
 4: 5,
 5: 4,
 6: 3,
 7: 0,
 8: 4,
 9: 0,
 10: 4,
 11: 1,
 12: 3,
 13: 2,
 14: 3,
 15: 2,
 16: 4,
 17: 2,
 18: 20,
 19: 20}

In [30]:
CostDict

{0: 1,
 1: 7,
 2: 12,
 3: 18,
 4: 16,
 5: 19,
 6: 6,
 7: 8,
 8: 16,
 9: 3,
 10: 5,
 11: 12,
 12: 4,
 13: 6,
 14: 14,
 15: 8,
 16: 5,
 17: 6,
 18: 361,
 19: 361}

In [31]:
ArcDict

{0: (1, 2),
 1: (1, 4),
 2: (1, 6),
 3: (1, 7),
 4: (1, 8),
 5: (2, 4),
 6: (2, 7),
 7: (2, 8),
 8: (3, 5),
 9: (3, 6),
 10: (4, 5),
 11: (4, 7),
 12: (4, 8),
 13: (5, 7),
 14: (5, 8),
 15: (6, 7),
 16: (6, 8),
 17: (7, 8),
 18: (2, 5),
 19: (3, 7)}

In [32]:
Delta_ij_p = {i:[0]*ArcCountWithDummyArcs for i in Paths}

for p in Paths:
    for i in ArcDict:    
            if ArcDict[i] in PathDict[p]:
                Delta_ij_p[p][i] = 1
# Delta_ij_p

In [33]:
NodeNodeAdjacencyMatrix

array([[0., 1., 0., 1., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [34]:
ReversedArcDict = {ArcDict[i]:i for i in range(ArcCountWithDummyArcs)}
ReversedArcDict

{(1, 2): 0,
 (1, 4): 1,
 (1, 6): 2,
 (1, 7): 3,
 (1, 8): 4,
 (2, 4): 5,
 (2, 7): 6,
 (2, 8): 7,
 (3, 5): 8,
 (3, 6): 9,
 (4, 5): 10,
 (4, 7): 11,
 (4, 8): 12,
 (5, 7): 13,
 (5, 8): 14,
 (6, 7): 15,
 (6, 8): 16,
 (7, 8): 17,
 (2, 5): 18,
 (3, 7): 19}

In [35]:
def PathModel():
    m1 = gp.Model("PathModel")
    
    Flow = m1.addVars(Paths,Commodities,name="flow")

    NetworkCost = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            NetworkCost += B_k_Dict[k] * PathCost[p] * Flow[p,k]
        for p in Paths:
            if p not in PathSetForCommodity[k]:
                m1.addConstr(Flow[p,k]==0)
    
    m1.setObjective(NetworkCost, GRB.MINIMIZE)

    ArcFlow = [0]*ArcCountWithDummyArcs
    InfoCollected = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    ArcFlow[i] += B_k_Dict[k] * Flow[p,k]
            InfoCollected += PathInfo[p]* Flow[p,k]
    
    CapacityConstraint = [0]*ArcCountWithDummyArcs
    
#     for i in range(ArcCountWithDummyArcs):
    CapacityConstraint = m1.addConstrs((ArcFlow[i] <= CapacityDict[i] for i in range(ArcCountWithDummyArcs)),name="Cap")
        
    InfoThresholdConstraint = m1.addConstr(InfoCollected>=RequiredInfo,name="InfoT")
    
    FlowConstraint = m1.addConstrs(((Flow.sum('*',k))==1 for k in Commodities),name="FlowPart")
            
    m1.write("C:\PhD IE\Optimization Models for Large Networks\path.lp")
#     print(InfoThresholdConstraint)
    m1.optimize()
    
    if m1.Status == GRB.OPTIMAL:
        OptimalFlow = m1.getAttr('X',Flow)
        ReducedCosts = m1.getAttr('RC',Flow)
#         print(ReducedCosts)
        Pi_ij = m1.getAttr('Pi',CapacityConstraint)
        ThresholdDual = m1.getConstrByName("InfoT")
        W = ThresholdDual.Pi
        Sigma = m1.getAttr('Pi',FlowConstraint)
        for k in Commodities:
            for p in PathSetForCommodity[k]:
                if OptimalFlow[p,k] != 0:
                    print("Flow on path",p,"by commodity",k,"=",OptimalFlow[p,k]*B_k_Dict[k],"units.")
    return Pi_ij,W,Sigma

In [36]:
def PathModel2():
    m1 = gp.Model("PathModel")
    
    BigM = sum(B_k)*10 + RequiredInfo*ArcCountWithDummyArcs
    
    Flow = m1.addVars(Paths,Commodities,name="flow")
    Y = m1.addVars(ArcCountWithDummyArcs,Commodities,name="info")

    NetworkCost = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            NetworkCost += B_k_Dict[k] * PathCost[p] * Flow[p,k]
        for p in Paths:
            if p not in PathSetForCommodity[k]:
                m1.addConstr(Flow[p,k]==0)
    
    m1.setObjective(NetworkCost, GRB.MINIMIZE)

    ArcFlow = [0]*ArcCountWithDummyArcs
    InfoCollected = 0

    for k in Commodities:
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    ArcFlow[i] += B_k_Dict[k] * Flow[p,k]
                    InfoCollected += InfoDict[i]* Y[i,k]
    
    CapacityConstraint = [0]*ArcCountWithDummyArcs
    
#     for i in range(ArcCountWithDummyArcs):
    CapacityConstraint = m1.addConstrs((ArcFlow[i] <= CapacityDict[i] for i in range(ArcCountWithDummyArcs)),name="Cap")
        
    InfoThresholdConstraint = m1.addConstr(InfoCollected>=RequiredInfo,name="InfoT")
    
    FlowCountOnArc = [0]*CommodityCount

    for k in Commodities:
        FlowCountOnArc[k-1] = [0]*ArcCountWithDummyArcs
        for p in PathSetForCommodity[k]:
            for i in range(len(Delta_ij_p[p])):
                if Delta_ij_p[p][i] == 1:
                    FlowCountOnArc[k-1][i] += BigM * Flow[p,k]
    
    InfoOnlyWithFlowConstraint = m1.addConstrs(((FlowCountOnArc[k-1][i] - Y[i,k]) >= 0 for i in range(ArcCountWithDummyArcs)for k in Commodities),name="InfoWithFlow")
    
    FlowConstraint = m1.addConstrs(((Flow.sum('*',k))==1 for k in Commodities),name="FlowPart")

    InfoRepeatConstraint = m1.addConstrs(((Y.sum(i,'*'))<=1 for i in range(ArcCountWithDummyArcs)),name="InfoSum")
    
    m1.write("C:\PhD IE\Optimization Models for Large Networks\path2.lp")
#     print(InfoThresholdConstraint)
    m1.optimize()
    
    if m1.Status == GRB.OPTIMAL:
        OptimalFlow = m1.getAttr('X',Flow)
        ReducedCosts = m1.getAttr('RC',Flow)
#         print(ReducedCosts)
        Pi_ij = m1.getAttr('Pi',CapacityConstraint)
        ThresholdDual = m1.getConstrByName("InfoT")
        W = ThresholdDual.Pi
        Sigma = m1.getAttr('Pi',FlowConstraint)
        U = m1.getAttr('Pi',InfoOnlyWithFlowConstraint)
        V = m1.getAttr('Pi',InfoRepeatConstraint)
        for k in Commodities:
            for p in PathSetForCommodity[k]:
                if OptimalFlow[p,k] != 0:
                    print("Flow on path",p,"by commodity",k,"=",OptimalFlow[p,k]*B_k_Dict[k],"units.")
    return Pi_ij,W,Sigma,U,V

In [37]:
IterationCounter = 0
Pi_ijWithArcNumber,W,Sigma = PathModel()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-01
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 46 rows, 25 columns and 55 nonzeros
Model fingerprint: 0xee3a40b3
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [7e+01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 46 rows and 25 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6390000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.639000000e+03
Flow on path 0 by commodity 1 = 15.0 units.
Flow on path 1 by commodity 2 = 10.0 units.
Flow on path 2 by commodity 3 = 6.0 units.
Flow on path 3 by commodity 4 = 6.0 units.
Flow on path 4 by commodity 5 =

In [38]:
# Pi_ijWithArcNumber, W, Sigma, U, V = PathModel2()

In [39]:
Pi_ij = {ArcDict[i]:0 for i in range(ArcCountWithDummyArcs)}

for i in range(ArcCountWithDummyArcs):
    Pi_ij[ArcDict[i]] = Pi_ijWithArcNumber[i]

print("Pi=",Pi_ij)
print("W=",W)
print("Sigma=",Sigma)

Pi= {(1, 2): 0.0, (1, 4): 0.0, (1, 6): 0.0, (1, 7): 0.0, (1, 8): 0.0, (2, 4): 0.0, (2, 7): 0.0, (2, 8): 0.0, (3, 5): 0.0, (3, 6): 0.0, (4, 5): 0.0, (4, 7): 0.0, (4, 8): 0.0, (5, 7): 0.0, (5, 8): 0.0, (6, 7): 0.0, (6, 8): 0.0, (7, 8): 0.0, (2, 5): 0.0, (3, 7): 0.0}
W= 0.0
Sigma= {1: 75.0, 2: 160.0, 3: 2166.0, 4: 72.0, 5: 2166.0}


In [40]:
ReversedArcDict = {ArcDict[i]:i for i in range(ArcCountWithDummyArcs)}
ReversedArcDict

{(1, 2): 0,
 (1, 4): 1,
 (1, 6): 2,
 (1, 7): 3,
 (1, 8): 4,
 (2, 4): 5,
 (2, 7): 6,
 (2, 8): 7,
 (3, 5): 8,
 (3, 6): 9,
 (4, 5): 10,
 (4, 7): 11,
 (4, 8): 12,
 (5, 7): 13,
 (5, 8): 14,
 (6, 7): 15,
 (6, 8): 16,
 (7, 8): 17,
 (2, 5): 18,
 (3, 7): 19}

In [41]:
def ReducedCostCompute(Pi_ijWithArcNumber,W,Sigma):
    ReducedCost = {k:[] for k in Commodities}
    
    for k in Commodities:
        ReducedCost[k] = [0]*ArcCountWithDummyArcs
    
    for k in Commodities:
        for i in range(ArcCountWithDummyArcs):
            for p in Paths:
                ReducedCost[k][i] += B_k_Dict[k]*CostDict[i]*Delta_ij_p[p][i] - B_k_Dict[k]*Pi_ijWithArcNumber[i]*Delta_ij_p[p][i] - W * InfoDict[i] #- Sigma[k]
            
    return ReducedCost

In [42]:
def BellmanFord(src,k,ReducedCost,Pi_ijWithArcNumber,W,Sigma):

    dist = [float("Inf")] * NodeCount
    prev = [-1] * NodeCount
    dist[src-1] = 0
    prev[src-1] = 0

    for _ in range(NodeCount - 1):

        for u in N:
            for v in NodeNodeAdjacencyList[u]:
#                 print(u,v,k,ReversedArcDict[(u,v)])
                if dist[u-1] != float("Inf") and dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]] < dist[v-1]:
                    dist[v-1] = dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]]
                    prev[v-1] = u-1

    for u in N:
        for v in NodeNodeAdjacencyList[u]:
            if dist[u-1] != float("Inf") and dist[u-1] + ReducedCost[k][ReversedArcDict[(u,v)]] < dist[v-1]:
                print("Graph contains negative weight cycle")
                return

    return dist, prev;
    

In [43]:
# Distance = [0]*NodeCount
# Prev = [0]*NodeCount


In [44]:
# Distance, Previous = BellmanFord(Origin[0],1)
# Destination

In [45]:
def MinReducedCostPath(Pi_ijWithArcNumber,W,Sigma):

    ReducedCost = ReducedCostCompute(Pi_ijWithArcNumber,W,Sigma) 
#     print("Reduced Costs:",ReducedCost)
    SP = [0]*CommodityCount

    for k in Commodities:

        Distance, Previous = BellmanFord(Origin[k-1],k,ReducedCost,Pi_ijWithArcNumber,W,Sigma)
        for i in range(len(Distance)):
            Distance[i] -= Sigma[k] 
#         print(Distance,Previous)
        
        if Distance[Destination[k-1]-1] < 0:

            SPTemp = [Destination[k-1]]
            T = Destination[k-1]-1

            while T!=Origin[k-1]-1:

                T = Previous[T]
                SPTemp.insert(0,T+1)

            SP[k-1] = SPTemp

    return SP

In [46]:
def Pricing(Pi_ijWithArcNumber,W,Sigma):
    ShortestPath = MinReducedCostPath(Pi_ijWithArcNumber,W,Sigma)
#     for k in range(CommodityCount):
#         ShortestPath[k] -= Sigma[k+1]
#     print("SP,O,D:",ShortestPath,Origin,Destination,"\n")
    NewPathCtr = 0
    PathLength = len(Paths)
    for k in range(CommodityCount):
#         VarCount = 0
        if ShortestPath[k] != 0:

#             for i in range(len(NodesInPath)):
#                 if NodesInPath[i] != ShortestPath[k]:
                    NewPathCtr += 1
                    Paths.append(PathLength-1+NewPathCtr)
                    PathSetForCommodity[k+1].append(PathLength-1+NewPathCtr)
                    NodesInPath[PathLength-1+NewPathCtr] = ShortestPath[k]
                    PathDict[PathLength-1+NewPathCtr] = []
                    PathCost[PathLength-1+NewPathCtr] = 0
                    PathInfo[PathLength-1+NewPathCtr] = 0
                    Delta_ij_p[PathLength-1+NewPathCtr] = [0]*ArcCountWithDummyArcs

                    for i in range(len(ShortestPath[k])-1):
                        PathDict[PathLength-1+NewPathCtr].append((ShortestPath[k][i],ShortestPath[k][i+1]))
                        PathCost[PathLength-1+NewPathCtr] += CostDict[ReversedArcDict[(ShortestPath[k][i],ShortestPath[k][i+1])]]
                        PathInfo[PathLength-1+NewPathCtr] += InfoDict[ReversedArcDict[(ShortestPath[k][i],ShortestPath[k][i+1])]]

                    for p in range(len(Paths)):
                        for i in range(len(ArcDict)):
                            if ArcDict[i] in PathDict[p]:
                                Delta_ij_p[p][i]=1
    return NewPathCtr

In [47]:
NewPathCtr = Pricing(Pi_ijWithArcNumber,W,Sigma)
# print(NewPathCtr, Paths)
IterationCounter = 1

while (NewPathCtr != 0) and IterationCounter < min(CommodityCount*CommodityCount,10*CommodityCount):
    IterationCounter += 1
    print("\nIteration",IterationCounter, "had", NewPathCtr,"paths added.\n\n")
    Pi_ijWithArcNumber,W,Sigma = PathModel()
#     print(Pi_ijWithArcNumber)
#     print(W)
#     print(Sigma)
    NewPathCtr = Pricing(Pi_ijWithArcNumber,W,Sigma)
#     print(NewPathCtr, Paths)
print("No new path added in iteration",IterationCounter,"\nSolution is optimal.")



Iteration 2 had 3 paths added.


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 58 rows, 40 columns and 91 nonzeros
Model fingerprint: 0x76186eec
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [7e+01, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+02]
Presolve removed 58 rows and 40 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2020000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.202000000e+03
Flow on path 0 by commodity 1 = 15.0 units.
Flow on path 1 by commodity 2 = 10.0 units.
Flow on path 5 by commodity 3 = 6.0 units.
Flow on path 3 by commodity 4 = 6.0 units.
Flow on path 4 by commodity 5 = 5.0 units.
Flow on path 7 by commodity 5 = 1.0 units.
No

In [48]:
PathDict

{0: [(4, 5)],
 1: [(3, 5)],
 2: [(3, 7)],
 3: [(4, 7)],
 4: [(2, 5)],
 5: [(3, 6), (6, 7)],
 6: [(4, 5), (5, 7)],
 7: [(2, 4), (4, 5)]}